In [1]:
import numpy as np
from timeit import timeit

# Numba

Numba is a nice little Python module that you can use to compile your code and make it faster. Different than other options (e.g. Cython), most of the time you don't have to change your code in anyway or do any external configurations or steps. All you need is to apply a *decorator* to a function. 

In [59]:
from numba import jit

# Pure Python vs. Numba
@jit
def exp_sum(X, base=2):
    s = 0
    for x in X:
        s += base ** x
    return s

# Numpy, for comparison
def exp_sum_np(X, base=2):
    return np.sum(base ** X)

X = np.random.rand(10000000)

print("Python (or Numba):", timeit(lambda: exp_sum(X), number=1))
print("Numpy:", timeit(lambda: exp_sum_np(X), number=1))
#timeit(lambda: exp_sum(X), setup=lambda: exp_sum(X), number=1)

Python (or Numba): 0.12949562199901266
Numpy: 0.10950372300067102


However, you cannot do everything with Numba. There are many things that it does not understand. For example, the function below will generate a warning and run at the same speed as pure Python, even though it works in the exact same way as the previous one. When Numba's `jit` finds something that is not supported, it gives a warning and falls back to slow mode. If you want, you can force it to work on fast mode only (`nopython`), which will generate an error in case some unsupported feature is used.

In [62]:
from numba import jit

# Something that Numba could not understand
@jit(nopython=True)
def exp_sum(X, base=2):
    return sum([base ** x for x in X])

X = np.random.rand(10000000)

print("Python (or Numba):", timeit(lambda: exp_sum(X), number=1))

TypingError: Failed in nopython mode pipeline (step: nopython frontend)
Untyped global name 'sum': cannot determine Numba type of <class 'builtin_function_or_method'>

File "<ipython-input-62-25ca816e2097>", line 6:
def exp_sum(X, base=2):
    return sum([base ** x for x in X])
    ^


Usually, **the simpler the code, the better for Numba**. So, if you want to use Numba to compile one of your functions, write it in the simplest possible way, potentially using pure Python.

Or, you can also use a subset of Numpy within Numba-compiled functions. This link shows exactly which Numpy functions (and their versions) are available for Numba: https://numba.pydata.org/numba-doc/dev/reference/numpysupported.html

In [65]:
# Broadcast 1
def broadcast1(X):
    n = X.shape[0]
    Z = np.empty((n, n))
    for i in range(n):        
        Z[i] = np.sqrt(np.sum((X[i] - X)**2, axis=-1))
    return Z

# Broadcast 1 + Numba
@jit(nopython=True)
def broadcast1_njit(X):
    n = X.shape[0]
    Z = np.empty((n, n))
    for i in range(n):        
        Z[i] = np.sqrt(np.sum((X[i] - X)**2, axis=-1))
    return Z

In [67]:
X = np.random.rand(10000,10)

print(timeit(lambda: broadcast1(X), setup=lambda: broadcast1(X), number=1))
print(timeit(lambda: broadcast1_njit(X), setup=lambda: broadcast1_njit(X), number=1))

4.456061894001323
3.25255720000132


## Decorators (Python)

We take a quick detour here to take a look at what a decorator is. In general, a decorator is *a function that preprocesses another function*. Let's look at a simple example.

In [10]:
def my_first_decorator(func):
    def modified_func():
        print(f">> Before [{func.__name__}]")
        original_return_value = func()
        print(f">> After [{func.__name__}]")
        return original_return_value
    return modified_func

@my_first_decorator
def a_simple_func():
    print("Just a simple function!")
    s = 0
    for i in range(1000):
        s += i ** 2
    return s
    
a_simple_func()

>> Before [a_simple_func]
Just a simple function!
>> After [a_simple_func]


332833500

A couple of important things to understand from the code above:
  * In Python, functions are objects just like everything else. So you can pass them as arguments to other functions, and access info about them such as `__name__`.
  * We defined a new function (`modified_func`) inside another function (`my_first_decorator`), which is totally fine. Notice how `modified_func` accesses `func` from the outside context. This is what is usually known as a *closure*.
  * The decorator must return a (possibly modified) function, so that the rest of the program can proceed normally. that is, when I call `a_simple_func` at the end, I'm actually calling `modified_func`, and not the original function anymore.

In [72]:
import my_module

In [73]:
my_module.my_function()

Hello, from the module!
